# **Json write ipynb**

- reference:  https://bleedai.com/introduction-to-pose-detection-and-basic-pose-classification/

In [1]:
import math
import cv2
import numpy as np
from time import time
import mediapipe as mp
import matplotlib.pyplot as plt
import json
from IPython.display import clear_output
import os
from PIL import Image
import mediapipe_helper
import ntpath
import glob

## **getGifAvgFps**

In [2]:
def getGifAvgFps(pil_ImageObject):
    """ Returns the average framerate of a PIL Image object """
    pil_ImageObject.seek(0)
    frames = duration = 0
    while True:
        try:
            frames += 1
            duration += pil_ImageObject.info['duration']
            pil_ImageObject.seek(pil_ImageObject.tell() + 1)
        except EOFError:
            return frames / duration * 1000
    return 24

## **Mediapipe init**

In [3]:
mp_pose = mp.solutions.pose
 
# Setting up the Pose function.
pose = mp_pose.Pose(static_image_mode=True, min_detection_confidence=0.6, model_complexity=2)

## **Mediapipe detect pose**

- This function performs pose detection on an image.
    - Args:
        - image: The input image with a prominent person whose pose landmarks needs to be detected.
        - pose: The pose setup function required to perform the pose detection.
        - display: A boolean value that is if set to true the function displays the original input image, the resultant image, 
       ' and the pose landmarks in 3D plot and returns nothing.
    - Returns:
        - output_image: The input image with the detected pose landmarks drawn.
        - landmarks: A list of detected landmarks converted into their original scale.

- Pose World landmarks
    - `x`: `-landmark[i].z`
    - `y`: `landmark[i].x` 
    - `z`: `-landmark[i].y`
    - `score`:  `landmark[i].visibility`
    - `name`: `name_list[i]`


In [4]:
def detectPose(image, pose, jsonObject, frameNum, display=True):
    clear_output(wait=True) # for clear console output
    # for json_object['frames']
    frameJsonObject = {
        "frameNum": frameNum,
        "keypoints": [
        ],
        "keypoints3D" : [
        ]
    }
    # Create a copy of the input image.
    output_image = image.copy()
    
    # Convert the image from BGR into RGB format.
    imageRGB = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    
    # Perform the Pose Detection.
    results = pose.process(imageRGB)

    if results.pose_world_landmarks:
        for i in range(0, 33):
            landmark = results.pose_world_landmarks.landmark
            frameJsonObject["keypoints3D"].append({'x': landmark[i].x,'y': landmark[i].y, 'z': landmark[i].z,'score':  landmark[i].visibility, "name": mediapipe_helper.mediapipe_names[i]})
            
    if results.pose_landmarks:
        # Draw Pose landmarks on the output image.
        for i in range(0, 33):
            landmark = results.pose_landmarks.landmark
            frameJsonObject["keypoints"].append({'x': landmark[i].x,'y': landmark[i].y, 'z': landmark[i].z,'score':  landmark[i].visibility, "name": mediapipe_helper.mediapipe_names[i]})
            
    jsonObject["frames"].append(frameJsonObject)
        
    # Return the output image and the found landmarks.
    return output_image

INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


In [5]:
def gif_to_mediapipe_json(fileName, maxFrameNum):
    FILENAME = os.path.join(os.path.dirname('__file__'),fileName)
    # only gif
    fps = getGifAvgFps(Image.open(FILENAME))
    jsonObject = {
        "fileName": fileName,
        "duration": 0,
        "ticksPerSecond": math.trunc(fps),
        "frames": [
        ]
    }
    frame_num = -1
    pose_video = mp_pose.Pose(static_image_mode=False, min_detection_confidence=0.5, model_complexity=1)
    video = cv2.VideoCapture(fileName)
    
    while video.isOpened():
        ok, frame = video.read()
        frame_num += 1
        if not ok or maxFrameNum < frame_num:
            break
        frame_height, frame_width, _ =  frame.shape
        frame = cv2.resize(frame, (int(frame_width * (640 / frame_height)), 640))
        frame = detectPose(frame, pose_video, jsonObject, frame_num, display=False)
        
    jsonObject['duration'] = frame_num -1
    video.release()
    
    return jsonObject
    
    

## **All gifs in '/sample' to json**

In [6]:
# json_object = gif_to_mediapipe_json("sample/mixamo_attack.gif", 100)
target_pattern = r"./sample/*.gif"
file_list = glob.glob(target_pattern)
file_list = [file.replace('\\', '/') for file in file_list]
for file in file_list:
     json_object = gif_to_mediapipe_json(file , 100)
     with open('./output/'+ ntpath.basename(file) +'.json', 'w') as f:
         json_string = json.dump(json_object, f, indent=2)    
